In [3]:
pwd

'C:\\Users\\Admin'

In [4]:
!pip install Keras
!pip install tensorflow

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True, vertical_flip=False)
test_datagen= ImageDataGenerator(rescale=1./255)

In [30]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_CF_zCnU3r3ob9Ykr5pZbPThZTaWYV5O-a6L4uGXFEF4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-0b7lfassq3idmq'
object_key = 'Data.zip'

streaming_body_8 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [31]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_8.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'C:\\Users\\Admin'

In [34]:
x_train = train_datagen.flow_from_directory("C:\Program Files\Data Collection\train\train",target_size=(100,100),class_mode='categorical',batch_size=75)
x_test = test_datagen.flow_from_directory("C:\Program Files\Data Collection\test\test",target_size=(100,100),class_mode='categorical',batch_size=75)

Found 13000 images belonging to 26 classes.
Found 2688 images belonging to 26 classes.


In [35]:
x_train.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [36]:
model = Sequential()

In [37]:
model.add(Convolution2D(32, (3,3), input_shape=(100,100,3),activation = 'relu')) #Feature map

In [38]:
model.add(MaxPooling2D(pool_size = (2,2))) #Pooled matrix

In [39]:
model.add(Flatten())

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 76832)             0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))

In [42]:
model.add(Dense(26,activation='softmax'))

In [43]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [44]:
len(x_train)

174

In [45]:
len(x_test)

36

In [46]:
model.fit_generator(x_train, steps_per_epoch=len(x_train), validation_data=x_test, validation_steps=len(x_test),epochs=10)

/tmp/wsuser/ipykernel_164/584640090.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train, steps_per_epoch=len(x_train), validation_data=x_test, validation_steps=len(x_test),epochs=10)


Epoch 1/10
174/174 [==============================] - 214s 1s/step - loss: 1.3648 - accuracy: 0.7414 - val_loss: 0.0235 - val_accuracy: 0.9996
Epoch 2/10
174/174 [==============================] - 214s 1s/step - loss: 0.0397 - accuracy: 0.9939 - val_loss: 0.0063 - val_accuracy: 1.0000
Epoch 3/10
174/174 [==============================] - 207s 1s/step - loss: 0.0086 - accuracy: 0.9984 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 5/10
174/174 [==============================] - 212s 1s/step - loss: 0.0058 - accuracy: 0.9992 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 6/10
174/174 [==============================] - 214s 1s/step - loss: 0.0035 - accuracy: 0.9994 - val_loss: 0.0024 - val_accuracy: 0.9989
Epoch 7/10
174/174 [==============================] - 216s 1s/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 8/10
174/174 [==============================] - 213s 1s/step - loss: 0.0058 - accuracy: 0.9989 - val_loss: 0.0012 - val_accuracy: 1.0000

In [47]:
model.save('HandSign.h5')

In [50]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 16.9 MB/s eta 0:00:01


In [62]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"3rHS2yjPVRZuSI3BERNZdd12z1795y1btiV6osUdMfdU"
}
client = APIClient(wml_credentials)

In [63]:
client = APIClient(wml_credentials)

In [68]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources']if item['entity']['name']== space_name)['metadata']['id'])

In [69]:
space_uid = guid_from_space_name(client,'imageclassification')
print("space UID=" + space_uid)

space UID=4ce1d0c0-8e85-4d4c-900c-4286c5a9f2c8


In [70]:
client.set.default_space(space_uid)

'SUCCESS'

In [71]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [154]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
print(software_spec_uid)

acd9c798-6974-5d2f-a657-ce06e986df4d


In [155]:
model_details = client.repository.store_model(model='specially-abled-model_new.tgz',meta_props={
     client.repository.ModelMetaNames.NAME:"CNN",
     client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
     client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                             )                                          
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [156]:
model_id

'520e2a02-e52b-4fc8-af7c-224398661308'

In [157]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'